In [1]:
%matplotlib widget
from util import get_path
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from extract_graph import generate_nx_graph, transform_list, generate_skeleton, generate_nx_graph_from_skeleton, from_connection_tab
from node_id import whole_movement_identification, second_identification
import ast
from plotutil import plot_t_tp1, compress_skeleton
from scipy import sparse
from sparse_util import dilate, zhangSuen
from realign import realign
from datetime import datetime,timedelta
from node_id import orient
import pickle
from matplotlib.widgets import CheckButtons
import scipy.io as sio
import imageio
from pymatreader import read_mat
import os
from matplotlib import colors
from random import choice
from experiment_class import Experiment,clean_exp_with_hyphaes
from hyphae_id import clean_and_relabel, get_mother, save_hyphaes

In [2]:
plate = 130
directory='//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE'
listdir=os.listdir(directory) 
list_dir_interest=[name for name in listdir if name.split('_')[-1]==f'Plate{0 if plate<10 else ""}{plate}']
ss=[name.split('_')[0] for name in list_dir_interest]
ff=[name.split('_')[1] for name in list_dir_interest]
dates_datetime=[datetime(year=int(ss[i][:4]),month=int(ss[i][4:6]),day=int(ss[i][6:8]),hour=int(ff[i][0:2]),minute=int(ff[i][2:4])) for i in range(len(list_dir_interest))]
dates_datetime.sort()
dates_datetime_chosen=dates_datetime[1:24]
dates = [f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}' for date in dates_datetime_chosen]

In [3]:
exp= Experiment(130)
exp.load(dates,local=False,raw=False)

In [4]:
exp_clean = clean_and_relabel(exp)

0.0
0.06795786612300374
0.13591573224600748
0.2038735983690112
0.27183146449201495
0.3397893306150187
0.4077471967380224
0.47570506286102615
0.5436629289840299
0.6116207951070336
0.6795786612300374
0.7475365273530411
0.8154943934760448
0.8834522595990486
0.9514101257220523
removing  1363  nodes
0.0
0.07547169811320754
0.1509433962264151
0.22641509433962265
0.3018867924528302
0.37735849056603776
0.4528301886792453
0.5283018867924528
0.6037735849056604
0.6792452830188679
0.7547169811320755
0.8301886792452831
0.9056603773584906
0.9811320754716981
removing  74  nodes


In [5]:
get_mother(exp_clean.hyphaes)

0.0
0.19076688286913393
0.38153376573826786
0.5723006486074017
0.7630675314765357
0.9538344143456696


In [6]:
hyphs,gr_inf = save_hyphaes(exp_clean)


In [12]:
exp_clean.save()

In [7]:
exp_clean.pickle_save()

In [194]:
exp_clean = pickle.load( open( f'Data/'+f"experiment_{plate}.pick", "rb" ) )

In [13]:
hyphae_with_degree4 = {}
for hyph in exp_clean.hyphaes:
    t0 = hyph.ts[-1]
    nodes,edges = hyph.get_nodes_within(t0)
    hyphae_with_degree4[hyph]=[]
    for node in nodes:
        if exp_clean.get_node(node).degree(t0)>=4:
            hyphae_with_degree4[hyph].append(node)

In [25]:
def resolve_ambiguity_two_ends(hyphaes,bottom_threshold=0.98):
    root_hyph={}
    hyphae_two_ends=[hyph for hyph in hyphaes if hyph.root.degree(hyph.ts[0])==1]
    print(len(hyphae_two_ends))
    to_remove = []
    x_boundaries = hyphaes[0].experiment.boundaries_x
    y_boundaries = hyphaes[0].experiment.boundaries_y
    for hyph in hyphae_two_ends:
        t0 = hyph.ts[0]
        if not hyph.root.pos(t0)[0]>=bottom_threshold*x_boundaries[1]:
            nodes,edges  = hyph.get_nodes_within(t0)
            mini = np.inf
            found = False
            for i,edge in enumerate(edges):
                if edge.end.degree(t0) >=4:
                    if edge.end.degree(t0)>=5:
                        print(hyph,edge.end, hyph.ts)
                    else:
                        next_edge = edges[i+1]
                        angle = np.cos((edge.orientation_end(t0,50)-next_edge.orientation_begin(t0,50))/360*2*np.pi)
                        if angle<mini:
                            found= True
                            maxi=angle
                            root_candidate = edge.end
            if found:
                root_hyph[hyph] = root_candidate
    ends = {hyph.end : hyph for hyph in hyphaes}
    for hyph in root_hyph.keys():
        if hyph.root in ends:
            ends[hyph.root].root = root_hyph[hyph]
        hyph.root = root_hyph[hyph]
    for hyph in hyphaes[0].experiment.hyphaes:
        hyph.update_ts()
    return(root_hyph)

In [23]:
for hyph in exp_clean.hyphaes:
    hyph.update_ts()

In [26]:
solve_two_ends = resolve_ambiguity_two_ends(exp_clean.hyphaes)

78
(Node(3679), Node(2354)) 2348 [4]
(Node(5931), Node(4056)) 5930 [12]
(Node(8897), Node(6619)) 8378 [21, 22]


In [15]:
hyphaes_interest = [hyph for hyph in hyphae_with_degree4.keys() if len(hyphae_with_degree4[hyph])>=1]

In [200]:
hyphae_two_ends=[hyph for hyph in exp_clean.hyphaes if hyph.root.degree(hyph.ts[0])==1]

In [83]:
weird = [hypha for hypha in hyphae_two_ends if hypha not in hyphaes_interest]

In [213]:
hyph = choice(hyphae_two_ends)
plt.close('all')

In [214]:
hyph.ts,hyph

([18, 19, 20, 21], Hyphae(7821,7832))

In [215]:
exp_clean.plot([18,20,21],[[hyph.root.label,hyph.end.label]]*3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [68]:
hyph.end.show_source_image(15)

OSError: The file //sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE/Plate13_20200627/InitialParameters.mat does not exist.